Install packages and import them

In [ ]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Scraping function depending on the year

In [173]:
def year_url(year):
  url="https://www.f1technical.net/f1db/cars/years/"+str(year)
  return url

In [178]:
#This function gives a dataframe containing all the F1 cars of the year chosen

def year_cars_dataframe(year):

  # First we get the url of the year we need
  url=year_url(year)

  #Get the HTML code of the interesting part of the page
  response=requests.get(url)
  soup=BeautifulSoup(response.text,"html.parser")
  content=soup.find_all("div", {"class": "content"})
  cars=content[2].findAll("tr")

#Creat a dictionaries list of F1 cars
  cars_list=[]
  for i in range(1,len(cars)):
    car={}
    car_td=cars[i].findAll("td")
    if car_td[0].a==None: # Fill All the missing information with "Unknown" 
      car["Chassis"]="Unknown"
    else:
      car["Chassis"]=car_td[0].text

    if car_td[1].img==None:
      car["Country"]="Unknown"
    else:
      car["Country"]=car_td[1].img.get('title')
    
    if car_td[2].a==None:
      car["Team"]="Unknown"
    else:
      car["Team"]=car_td[2].text

    car["Year"]=year

    cars_list.append(car)
    
  #Convert the list to a Data Frame
  df=pd.DataFrame(cars_list,columns=["Chassis","Country","Team","Year"])
  return df

In [189]:
#Scrap All the years
dfList=[]
for year in range(1950,2023):
  dfList.append(year_cars_dataframe(year))

In [197]:
#Concatenate all the years in one Dataframe
final_df=pd.concat(dfList)

In [203]:
#Convert the Dataframe to a csv and xlsx files
final_df.to_csv("All_F1_Cars_Of_The_History.csv",index=False,encoding="utf-8")
final_df.to_excel("All_F1_Cars_Of_The_History.xlsx",index=False,encoding="utf-8")